In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Modified on 2025-05-18

@Updated by ChatGPT for conditional checkpoint saving
"""

from __future__ import print_function
from tensorflow import keras
import numpy as np
import pandas as pd
import os

# 시드 고정
np.random.seed(813306)

# 데이터 로딩 함수
def readucr(filename):
    data = np.loadtxt(filename, delimiter=',')
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y

# 사용자 정의 콜백: 특정 epoch 이후 val_accuracy 향상 시에만 저장
class ConditionalCheckpoint(keras.callbacks.ModelCheckpoint):
    def __init__(self, filepath, start_epoch=1500, **kwargs):
        super().__init__(filepath, **kwargs)
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch + 1 >= self.start_epoch:
            super().on_epoch_end(epoch, logs)

# 데이터셋 목록
flist = ['Adiac']

# 반복 실행
for fname in flist:
    # 데이터 로딩
    x_train, y_train = readucr(fname + '/' + fname + '_TRAIN')
    x_test, y_test = readucr(fname + '/' + fname + '_TEST')

    # 클래스 수와 정규화
    nb_classes = len(np.unique(y_test))
    y_train = (y_train - y_train.min()) / (y_train.max() - y_train.min()) * (nb_classes - 1)
    y_test = (y_test - y_test.min()) / (y_test.max() - y_test.min()) * (nb_classes - 1)
    batch_size = min(x_train.shape[0] // 10, 16)

    # 원-핫 인코딩
    Y_train = keras.utils.to_categorical(y_train, nb_classes)
    Y_test = keras.utils.to_categorical(y_test, nb_classes)

    # 입력 정규화
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean) / x_train_std
    x_test = (x_test - x_train_mean) / x_train_std

    # 모델 정의 (MLP-XII)
    x = keras.layers.Input(x_train.shape[1:])
    y = keras.layers.Dense(500, activation='linear')(x)
    y = keras.layers.Dense(500, activation='linear')(y)
    y = keras.layers.Dense(500, activation='linear')(y)
    y = keras.layers.Dense(500, activation='linear')(y)
    out = keras.layers.Dense(nb_classes, activation='softmax')(y)

    model = keras.models.Model(inputs=x, outputs=out)

    # 컴파일
    optimizer = keras.optimizers.Adadelta()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # 콜백 정의
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor='loss', factor=0.5, patience=200, min_lr=0.1
    )

    os.makedirs('saved_models', exist_ok=True)
    checkpoint_callback = ConditionalCheckpoint(
        filepath='saved_models/MLP_epoch_{epoch:04d}_valacc_{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max',
        start_epoch=1500
    )

    # 학습
    nb_epochs = 2000
    hist = model.fit(
        x_train, Y_train,
        batch_size=10,
        epochs=nb_epochs,
        verbose=1,
        validation_data=(x_test, Y_test),
        callbacks=[reduce_lr, checkpoint_callback]
    )

    # 로깅 저장
    log_df = pd.DataFrame(hist.history)
    log_df.to_csv('training_log.csv', index=False)

    # 최종 모델 저장
    os.makedirs('save_model/h5', exist_ok=True)
    model.save('save_model/h5/mlp_model.h5', save_format='h5')

    # 모델 불러오기 예시
    mlp_model12 = keras.models.load_model('save_model/h5/mlp_model.h5')



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\KJH\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\KJH\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\KJH\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\KJH\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\KJH\anaconda3\lib\site-packages\ipyke

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\KJH\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\KJH\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\KJH\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\KJH\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\KJH\anaconda3\lib\site-packages\ipyke

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\KJH\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\KJH\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\KJH\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\KJH\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\KJH\anaconda3\lib\site-packages\ipyke

...

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7279 - loss: 0.5591 - val_accuracy: 0.7162 - val_loss: 0.7250 - learning_rate: 0.0010
Epoch 1907/2000
56/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7167 - loss: 0.5438
Epoch 1907: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7163 - loss: 0.5450 - val_accuracy: 0.7047 - val_loss: 0.7398 - learning_rate: 0.0010
Epoch 1908/2000
54/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7374 - loss: 0.5508
Epoch 1908: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7345 - loss: 0.5521 - val_accuracy: 0.7015 - val_loss: 0.7582 - learning_rate: 0.0010
Epoch 1909/2000
54/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7000 - loss: 0.5710
Epoch 1909: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7014 - loss: 0.5700 - val_accuracy: 0.7031 - val_loss: 0.7905 - learning_rate: 0.0010
Epoch 1910/

Epoch 1930/2000
56/59 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7034 - loss: 0.5504
Epoch 1930: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7037 - loss: 0.5511 - val_accuracy: 0.6966 - val_loss: 0.7987 - learning_rate: 0.0010
Epoch 1931/2000
58/59 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7069 - loss: 0.5641
Epoch 1931: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7072 - loss: 0.5640 - val_accuracy: 0.6900 - val_loss: 0.8095 - learning_rate: 0.0010
Epoch 1932/2000
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6906 - loss: 0.5805
Epoch 1932: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6909 - loss: 0.5802 - val_accuracy: 0.6900 - val_loss: 0.8154 - learning_rate: 0.0010
Epoch 1933/2000
58/59 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6961 - loss: 0.5818 
Epoch 1933: val_accuracy did not improve from 0.71615
59/5

56/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7267 - loss: 0.5523
Epoch 1953: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7260 - loss: 0.5529 - val_accuracy: 0.6966 - val_loss: 0.7994 - learning_rate: 0.0010
Epoch 1954/2000
58/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6904 - loss: 0.5631
Epoch 1954: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6911 - loss: 0.5630 - val_accuracy: 0.6884 - val_loss: 0.8548 - learning_rate: 0.0010
Epoch 1955/2000
55/59 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7347 - loss: 0.5432
Epoch 1955: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7332 - loss: 0.5446 - val_accuracy: 0.6884 - val_loss: 0.8361 - learning_rate: 0.0010
Epoch 1956/2000
56/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7285 - loss: 0.5753
Epoch 1956: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7326 - loss: 0.5604 - val_accuracy: 0.6884 - val_loss: 0.8598 - learning_rate: 0.0010
Epoch 1977/2000
54/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7016 - loss: 0.5663
Epoch 1977: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7031 - loss: 0.5657 - val_accuracy: 0.6917 - val_loss: 0.8271 - learning_rate: 0.0010
Epoch 1978/2000
52/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7790 - loss: 0.5139
Epoch 1978: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7704 - loss: 0.5201 - val_accuracy: 0.6966 - val_loss: 0.7749 - learning_rate: 0.0010
Epoch 1979/2000
54/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7212 - loss: 0.5341
Epoch 1979: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7200 - loss: 0.5369 - val_accuracy: 0.7047 - val_loss: 0.7897 - learning_rate: 0.0010
Epoch 1980/200

Epoch 2000/2000
52/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7196 - loss: 0.5468
Epoch 2000: val_accuracy did not improve from 0.71615
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7185 - loss: 0.5486 - val_accuracy: 0.7015 - val_loss: 0.7839 - learning_rate: 0.0010


In [5]:
import numpy as np
import pandas as pd
from tensorflow import keras

def readucr(filename):
    data = np.loadtxt(filename, delimiter=',')
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y

# 1. 데이터셋
fname = 'Adiac'
x_train, _ = readucr(f'{fname}/{fname}_TRAIN')   # ✅ 학습 기준 정규화
x_test, y_test = readucr(f'{fname}/{fname}_TEST')

# 2. 라벨 정수화
y_true = y_test.astype(int)

# 3. 정규화 (x_train 기준으로 x_test 정규화)
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_test = (x_test - x_train_mean) / x_train_std

# 4. 모델 불러오기
model = keras.models.load_model('saved_models/MLP_epoch_1906_valacc_0.7162.h5')

# 5. 예측
y_pred_prob = model.predict(x_test)
y_pred = y_pred_prob.argmax(axis=-1)

# 6. 정확도 계산
match = (y_pred == y_true)
accuracy = np.mean(match)

print(f"✅ 정확도: {accuracy * 100:.2f}%")
print(f"✅ 일치 수: {np.sum(match)} / {len(y_true)}")

# 7. 결과 저장
df_result = pd.DataFrame({
    'Actual': y_true,
    'Predicted': y_pred,
    'Match': match
})
df_result.to_csv('MLP_epoch_1906_prediction_results.csv', index=False)
print("📁 예측 결과를 'MLP_epoch_1906_prediction_results.csv'에 저장했습니다.")


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
✅ 정확도: 71.62%
✅ 일치 수: 439 / 613
📁 예측 결과를 'MLP_epoch_1906_prediction_results.csv'에 저장했습니다.
